# Mangrove Kandel Backtester - Samples


## Engine


### 0. Imports


In [1]:
import math
import json
import numpy as np
import pandas as pd

from src.kandel_backtester import KandelBacktester

In [2]:
with open("config.json") as f:
    config = json.load(f)


window = config["window"]
quote = config["quote"]
base = config["base"]
vol_mult = config["vol_mult"]
n_points = config["n_points"]
step_size = config["step_size"]
vol_threshold_window = config["vol_threshold_window"]
samples_length = config["samples_length"]

del config

### 1. Preprocessing


#### Load data


In [3]:
prices_df = pd.read_csv("data/binance/ETHUSDC-2024-01>08_1s.csv", header=0, index_col=0)
prices_df.index = pd.to_datetime(prices_df.index, unit="s", utc=True)

In [4]:
prices_df["daily_log_return"] = np.log(prices_df["price"] / prices_df["price"].shift(1))
prices_df["vol"] = (
    prices_df["daily_log_return"].rolling(vol_threshold_window * 3600).std() * np.sqrt(vol_threshold_window*3600)
)
prices_df["vol"] = prices_df["vol"].fillna(0)

prices_df = prices_df.drop(columns=["daily_log_return"])

#### Create samples


In [5]:
samples_length_s = (samples_length * 24 + window) * 3600

# Remove days used for volatility calculation
prices_df_trimmed = prices_df.iloc[(vol_threshold_window + window) * 3600 :]

prices_splitted = []
vol_splitted = []

for i in range(samples_length + math.ceil(window / 24)):
    prices_splitted += [
        prices_df_trimmed["price"][j : j + samples_length_s]
        for j in range(i * 3600 * 24, len(prices_df_trimmed["price"]), samples_length_s)
    ][:-1]
    vol_splitted += [
        prices_df_trimmed["vol"][j : j + samples_length_s]
        for j in range(i * 3600 * 24, len(prices_df_trimmed["vol"]), samples_length_s)
    ][:-1]

del prices_df_trimmed

### 2. Run backtester

In [6]:
from src.kandel_v2 import Kandel, KandelConfig

kandel = Kandel(
    config=KandelConfig(
        window=window,
        initial_quote=quote,
        initial_base=base,
        vol_mult=vol_mult,
        n_points=n_points,
        step_size=step_size,
        vol_threshold_window=vol_threshold_window,
    ),
    spot_price=prices_splitted[0][0],
    vol=vol_splitted[0][0],
)

/var/folders/kn/wgl1sqns1bxgynxc599_5y9c0000gp/T/ipykernel_30528/500993707.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  spot_price=prices_splitted[0][0],
/var/folders/kn/wgl1sqns1bxgynxc599_5y9c0000gp/T/ipykernel_30528/500993707.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vol=vol_splitted[0][0],


KeyError: 'initial_quote'